<a href="https://colab.research.google.com/github/gptix/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Jud_Taylor_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [0]:
# Get data set.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-11-05 16:10:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.04s   

2019-11-05 16:10:45 (460 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
# Make data useable. 

# Add column headers. Replace "?" with "NaN".
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

df = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers,
                 na_values="?")

# convert yes/no to numeric (no=0)
df = df.replace({'y': 1, 'n': 0})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
republican = df[df['party']=='republican']
republican.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
democrat = df[df['party']=='democrat']
democrat.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


##Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
###Two Sample t-test/

1) Null Hypothesis: There is no difference between average voting rates (levels of support) for a bill bill between Democrats and Republicans in the house of representatives.

$\bar{x}_{1} == \bar{x}_{2}$

Where $\bar{x}_{1}$ is the mean of Republican votes and $\bar{x}_{2}$ is the mean of Democrat votes.

2) Alternative Hypothesis: 

$\bar{x}_{1} \neq \bar{x}_{2}$

Levels of support between the two parties differ. 

3) 99% Confidence Level

In [0]:
# Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

confidence_int = .01
#CHANGE

# Run ttest from scipy by specifying two series (specific columns in a specific dataframes).
# ttest_ind(republican['water-project'], democrat['water-project'])
# NaN's exist.

ttest_ind(republican['water-project'], democrat['water-project'], nan_policy='omit')

# 4) T-statistic: .089

# 5) P-value: .929

pvalue=0.9291556823993485

In [0]:
issues = ['handicapped-infants','water-project', 'budget',
          'physician-fee-freeze', 'el-salvador-aid', 'religious-groups',
          'anti-satellite-ban', 'aid-to-contras','mx-missile','immigration',
          'synfuels', 'education', 'right-to-sue','crime','duty-free',
          'south-africa']
                          
def test_issue(issue):
  t_stat, p_val = ttest_ind(republican[issue], democrat[issue], nan_policy='omit')
  return (print(issue + " --- p_val: " + str(p_val) ))

test_issue('handicapped-infants')


handicapped-infants --- p_val: 1.613440327937243e-18


In [0]:
def test_issues(issues):
  for i in issues:
    test_issue(i)

test_issues(issues)

handicapped-infants --- p_val: 1.613440327937243e-18
water-project --- p_val: 0.9291556823993485
budget --- p_val: 2.0703402795404463e-77
physician-fee-freeze --- p_val: 1.994262314074344e-177
el-salvador-aid --- p_val: 5.600520111729011e-68
religious-groups --- p_val: 2.3936722520597287e-20
anti-satellite-ban --- p_val: 8.521033017443867e-31
aid-to-contras --- p_val: 2.82471841372357e-54
mx-missile --- p_val: 5.03079265310811e-47
immigration --- p_val: 0.08330248490425066
synfuels --- p_val: 1.5759322301054064e-15
education --- p_val: 1.8834203990450192e-64
right-to-sue --- p_val: 1.2278581709672758e-34
crime --- p_val: 9.952342705606092e-47
duty-free --- p_val: 5.997697174347365e-32
south-africa --- p_val: 3.652674361672226e-11


In [0]:
def get_pval(issue):
  pval, probability = ttest_ind(republican[issue], democrat[issue], nan_policy='omit')
  return pval

get_pval('water-project')

0.08896538137868286

In [0]:
def reject_yn (issue, conf_int):
  return (get_pval(issue) < (1 - conf_int))

In [0]:
reject_yn('water-project', 0.01)

True

In [0]:
for i in issues:
  print(" --- Reject null hypothesis for issue: " + i + ": " + str(reject_yn(i, 0.01)))

 --- Reject null hypothesis for issue: handicapped-infants: True
 --- Reject null hypothesis for issue: water-project: True
 --- Reject null hypothesis for issue: budget: True
 --- Reject null hypothesis for issue: physician-fee-freeze: False
 --- Reject null hypothesis for issue: el-salvador-aid: False
 --- Reject null hypothesis for issue: religious-groups: False
 --- Reject null hypothesis for issue: anti-satellite-ban: True
 --- Reject null hypothesis for issue: aid-to-contras: True
 --- Reject null hypothesis for issue: mx-missile: True
 --- Reject null hypothesis for issue: immigration: False
 --- Reject null hypothesis for issue: synfuels: True
 --- Reject null hypothesis for issue: education: False
 --- Reject null hypothesis for issue: right-to-sue: False
 --- Reject null hypothesis for issue: crime: False
 --- Reject null hypothesis for issue: duty-free: True
 --- Reject null hypothesis for issue: south-africa: True


In [0]:
def find_reject_nulls(issues):
  print("issues with rejected null hypotheses:")
  for i in issues:
    if (reject_yn(i, 0.01)):
      print(i)

In [0]:
find_reject_nulls(issues)

issues with rejected null hypotheses:
handicapped-infants
water-project
budget
anti-satellite-ban
aid-to-contras
mx-missile
synfuels
duty-free
south-africa


In [0]:
issues_with_rejected_null_hypotheses = ['handicapped-infants', 
                                        'water-project',
                                        'budget',
                                        'anti-satellite-ban',
                                        'aid-to-contras',
                                        'mx-missile',
                                        'synfuels',
                                        'duty-free',
                                        'south-africa']

In [0]:
republican_count = len(republican)
democrat_count = len(democrat)
print("republicans: " + str(republican_count))
print("democrats: " + str(democrat_count))

republicans: 168
democrats: 267


In [0]:
def percent_voted(issue):
  d_vote_percent = 100 * democrat[issue].sum()/democrat_count
  r_vote_percent = 100 * republican[issue].sum()/republican_count
  if (d_vote_percent > r_vote_percent):
    print("Null Hypothesis rejected: Democrats voted greater percentage ("
           + str(round (d_vote_percent,1)) + "%) than Republicans (" +
          str(round(r_vote_percent,1)) + "%) on: " + issue)
  else:
    print("Null Hypothesis rejected: Republicans voted greater percentage than Democrats on: " + issue)
  # return d_vote_percent

#test
percent_voted('water-project')

Null Hypothesis rejected: Democrats voted greater percentage (44.9%) than Republicans (44.6%) on: water-project


In [0]:
for issue in issues_with_rejected_null_hypotheses:
  percent_voted(issue)

Null Hypothesis rejected: Democrats voted greater percentage (58.4%) than Republicans (18.5%) on: handicapped-infants
Null Hypothesis rejected: Democrats voted greater percentage (44.9%) than Republicans (44.6%) on: water-project
Null Hypothesis rejected: Democrats voted greater percentage (86.5%) than Republicans (13.1%) on: budget
Null Hypothesis rejected: Democrats voted greater percentage (74.9%) than Republicans (23.2%) on: anti-satellite-ban
Null Hypothesis rejected: Democrats voted greater percentage (81.6%) than Republicans (14.3%) on: aid-to-contras
Null Hypothesis rejected: Democrats voted greater percentage (70.4%) than Republicans (11.3%) on: mx-missile
Null Hypothesis rejected: Democrats voted greater percentage (48.3%) than Republicans (12.5%) on: synfuels
Null Hypothesis rejected: Democrats voted greater percentage (59.9%) than Republicans (8.3%) on: duty-free
Null Hypothesis rejected: Democrats voted greater percentage (64.8%) than Republicans (57.1%) on: south-africa


## This answers both
* Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
* Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01


## Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

In [0]:
def find_p_gt_ten_percent(issues):
  print("Issues with p > 0.1:")
  for i in issues:
    t_stat, p_val =  ttest_ind(republican[issue], democrat[issue], nan_policy='omit')
    if (p_val > 0.1):
      print(i)

#Stretch goals:

Refactor your code into functions so it's easy to rerun with arbitrary variables.

In [0]:
## Analyze two isomorphic data frames, data column by data column.
## Produce a list of dictionaries holding pertinant results.
def analyze_dfs(df1, df2, cols_to_examine, conf_interval):
  col_dicts = []
 
  for col in cols_to_examine:
    column_dict = {'column' : col}

    t_stat, p_val = ttest_ind(df1[col], df2[col], nan_policy='omit')
    data = { 't statistic' : t_stat,
            'p value' : p_val}
    if (p_val < conf_interval):
      data['reject null hypo'] = True
    else:
      data['reject null hypo'] = False
      
    column_dict['data'] = data
    col_dicts.append(column_dict)
    
  rslts = {'column dicts' : col_dicts, "confidence interval" : conf_interval}

  return rslts

analysis = analyze_dfs(republican, democrat, issues, 0.01)

analysis

{'column dicts': [{'column': 'handicapped-infants',
   'data': {'p value': 1.613440327937243e-18,
    'reject null hypo': True,
    't statistic': -9.205264294809222}},
  {'column': 'water-project',
   'data': {'p value': 0.9291556823993485,
    'reject null hypo': False,
    't statistic': 0.08896538137868286}},
  {'column': 'budget',
   'data': {'p value': 2.0703402795404463e-77,
    'reject null hypo': True,
    't statistic': -23.21277691701378}},
  {'column': 'physician-fee-freeze',
   'data': {'p value': 1.994262314074344e-177,
    'reject null hypo': True,
    't statistic': 49.36708157301406}},
  {'column': 'el-salvador-aid',
   'data': {'p value': 5.600520111729011e-68,
    'reject null hypo': True,
    't statistic': 21.13669261173219}},
  {'column': 'religious-groups',
   'data': {'p value': 2.3936722520597287e-20,
    'reject null hypo': True,
    't statistic': 9.737575825219457}},
  {'column': 'anti-satellite-ban',
   'data': {'p value': 8.521033017443867e-31,
    'reject